In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 0. Loading the Homework dataset

In [2]:
train_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
val_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
train_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Q1. Downloading the data

Read the data for January. How many columns are there?

In [4]:
train_data.shape[1] # 19 columns

19

**Response**: 19 columns ✔

# Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [5]:
# Obtaining the duration (in minutes) of every trip in the dataset
train_data['duration'] = train_data['tpep_dropoff_datetime'] - train_data['tpep_pickup_datetime']
train_data['duration'] = train_data['duration'].apply(lambda td: td.total_seconds() / 60)

In [6]:
train_data['duration'].std() # Std of duration = 42.59 minutes

42.594351241920904

**Response**: 42.59 minutes ✔

# Q3. Dropping outliers

Next, we need to check the distribution of the ´duration´ variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
train_data['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
train_data_filtered = train_data[(train_data['duration'] >= 1) & (train_data['duration'] <= 60)]
records_left = train_data_filtered.shape[0]/train_data.shape[0]
print(f'{round(records_left*100,2)}% left after the removal of outliers ✔')

98.12% left after the removal of outliers ✔


# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [10]:
# Choosing the variables of interest
categorical = ['PULocationID', 'DOLocationID']

# Transforming the chosen categorical variables into strings
train_data_filtered[categorical] = train_data_filtered[categorical].astype(str)

train_data_filtered[categorical].head()

/tmp/ipykernel_11943/2544121616.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_filtered[categorical] = train_data_filtered[categorical].astype(str)


,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


In [11]:
# Turning the columns into a dictionary
train_dicts = train_data_filtered[categorical].to_dict(orient='records')
train_dicts[0:5]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [12]:
# Apply One Hot Encoding for categorical variables (DictVectorizer) and fit
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train.shape # 515 columns

(3009173, 515)

**Response**: 515 columns ✔

# Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [14]:
# Target variable
target = 'duration'
y_train = train_data_filtered[target].values

# Training the linear Regresion with the Train set (without parameters)
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
# Making predictions with the X train set
y_pred = lr.predict(X_train)

# MSE for the Train set (7.64 minutes error in average)
mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

**Response**: 7.64 minutes ✔

# Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [16]:
# Preprocessing the validation dataset

# Obtaining the duration (in minutes) of every trip in the validation dataset
val_data['duration'] = val_data['tpep_dropoff_datetime'] - val_data['tpep_pickup_datetime']
val_data['duration'] = val_data['duration'].apply(lambda td: td.total_seconds() / 60)

# Filtering the val dataset
val_data_filtered = val_data[(val_data['duration'] >= 1) & (val_data['duration'] <= 60)]

In [17]:
# Preparing the val data for the model

# Choosing the variables of interest
categorical = ['PULocationID', 'DOLocationID']

# Transforming the chosen categorical variables into strings
val_data_filtered[categorical] = val_data_filtered[categorical].astype(str)

# Target variable
target = 'duration'
y_val = val_data_filtered[target].values

/tmp/ipykernel_11943/160166956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data_filtered[categorical] = val_data_filtered[categorical].astype(str)


In [18]:
# Fiting the model

# Turning the columns into a dictionary
val_dicts = val_data_filtered[categorical].to_dict(orient='records')

# Apply One Hot Encoding for categorical variables (DictVectorizer) and fit
X_val = dv.fit_transform(val_dicts)

# Training the linear Regresion with the Val set (without parameters)
lr = LinearRegression()
lr.fit(X_val, y_val)

LinearRegression()

In [19]:
# RMSE for validation

# Making predictions with the X train set
y_pred_val = lr.predict(X_val)

# MSE for the Train set (7.64 minutes error in average)
mean_squared_error(y_val, y_pred_val, squared=False)

7.778947591715993

**Response**: 7.77 minutes ✔